In [1]:
##### This file is for `training` the model. The steps are as follows:

# 1. `Import Dependencies`
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from keras.layers import Conv1D

In [9]:
# 2. `Setup Folders` for Collection
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('../NPY_FILES_1/') 
    
# Actions that we try to detect
actions = np.array(['ako', 'bakit', 'hi', 'hindi', 'ikaw', 'kamusta','maganda','magandang umaga', 'oo', 'salamat', 'F', 'L', 'P','N', 'O', 'none'])

# number of videos we want to use for training
no_sequences = 959 # edit this to change the number of videos used for training

# Videos are going to be 30 frames in length
sequence_length = 30

In [11]:
# 3. `Preprocess` Data and `Create Labels and Features`
label_map = {label:num for num, label in enumerate(actions)}
print(label_map)

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        print('action: ', action, 'sequence: ', sequence)
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
print('np.array(sequences).shape: ', np.array(sequences).shape)
print('np.array(labels).shape: ',np.array(labels).shape)

y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print('y_test shape:', y_test.shape)

{'ako': 0, 'bakit': 1, 'hi': 2, 'hindi': 3, 'ikaw': 4, 'kamusta': 5, 'maganda': 6, 'magandang umaga': 7, 'oo': 8, 'salamat': 9, 'F': 10, 'L': 11, 'P': 12, 'N': 13, 'O': 14, 'none': 15}
action:  ako sequence:  0
action:  ako sequence:  1
action:  ako sequence:  10
action:  ako sequence:  100
action:  ako sequence:  101
action:  ako sequence:  102
action:  ako sequence:  103
action:  ako sequence:  104
action:  ako sequence:  105
action:  ako sequence:  106
action:  ako sequence:  107
action:  ako sequence:  108
action:  ako sequence:  109
action:  ako sequence:  11
action:  ako sequence:  110
action:  ako sequence:  111
action:  ako sequence:  112
action:  ako sequence:  113
action:  ako sequence:  114
action:  ako sequence:  115
action:  ako sequence:  116
action:  ako sequence:  117
action:  ako sequence:  118
action:  ako sequence:  119
action:  ako sequence:  12
action:  ako sequence:  120
action:  ako sequence:  121
action:  ako sequence:  122
action:  ako sequence:  123
action:  a

In [12]:
# 4. Build and `Train` LSTM Neural Network
log_dir = os.path.join('Logs')

# Model design and implementation
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(30,258)))
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 28, 64)            49600     
                                                                 
 lstm (LSTM)                 (None, 28, 64)            33024     
                                                                 
 lstm_1 (LSTM)               (None, 28, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                5

In [14]:
# train model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger
import csv

filename = 'training.log'

# Check if file does not exist
if not os.path.isfile(filename):
    # Create the file
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(["Epoch", "Loss", "Accuracy", "Val_Loss", "Val_Accuracy"])

csv_logger = CSVLogger('training.log')

tb_callback = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

model.fit(X_train, y_train, epochs=2000, batch_size=64, validation_data=(X_test, y_test), callbacks=[csv_logger, checkpoint, early_stopping, reduce_lr,tb_callback])

Epoch 1/2000
431/432 [============================>.] - ETA: 0s - loss: 2.5163 - categorical_accuracy: 0.1301
Epoch 1: val_loss improved from inf to 2.22254, saving model to model-001.h5
432/432 [==============================] - 23s 42ms/step - loss: 2.5151 - categorical_accuracy: 0.1304 - val_loss: 2.2225 - val_categorical_accuracy: 0.1947 - lr: 0.0010
Epoch 2/2000
  3/432 [..............................] - ETA: 17s - loss: 2.2488 - categorical_accuracy: 0.1771

c:\Users\iceon\anaconda3\envs\fslenv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


431/432 [============================>.] - ETA: 0s - loss: 2.0243 - categorical_accuracy: 0.2775
Epoch 2: val_loss improved from 2.22254 to 1.78945, saving model to model-002.h5
432/432 [==============================] - 17s 40ms/step - loss: 2.0244 - categorical_accuracy: 0.2773 - val_loss: 1.7895 - val_categorical_accuracy: 0.3411 - lr: 0.0010
Epoch 3/2000
432/432 [==============================] - ETA: 0s - loss: 1.6245 - categorical_accuracy: 0.4102
Epoch 3: val_loss improved from 1.78945 to 1.41565, saving model to model-003.h5
432/432 [==============================] - 17s 40ms/step - loss: 1.6245 - categorical_accuracy: 0.4102 - val_loss: 1.4156 - val_categorical_accuracy: 0.4974 - lr: 0.0010
Epoch 4/2000
431/432 [============================>.] - ETA: 0s - loss: 1.2674 - categorical_accuracy: 0.5510
Epoch 4: val_loss improved from 1.41565 to 1.23005, saving model to model-004.h5
432/432 [==============================] - 17s 40ms/step - loss: 1.2682 - categorical_accuracy: 0.55

In [ ]:
# 5. `Save/Load` Weights
modelName = 'fsl.h5' # edit this to change the model name
model.save(modelName)

In [ ]:
# 6. `Evaluate` Model using Confusion Matrix and Accuracy
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print(multilabel_confusion_matrix(ytrue, yhat))
print(accuracy_score(ytrue, yhat))


In [ ]:
# 7. Save the Model to tfjs

def modelToTFJS(model, modelName):
    import tensorflowjs as tfjs
    os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
    
    # numpy expects the following
    np.bool = np.bool_
    np.object = object

    model.load_weights(modelName)
    output_dir = '' # edit this to change the output directory
    tfjs.converters.save_keras_model(model, output_dir)
    
# modelToTFJS(model, modelName)
